# This notebook contains step-by-step instructions for using Dropseqtools 1.13 (computational protocol 1.0.1)

- The following notebook contains instructions for aligning reads generated from SeqWell/DropSeq protocols. Much of this is taken directly from the McCarroll Lab's [alignment cookbook](http://mccarrolllab.com/wp-content/uploads/2016/03/Drop-seqAlignmentCookbookv1.2Jan2016.pdf), with copied items being *italicized*. Here, we will run Dropseqtools on data from the first batch of data on mouse retina from the Macosko et al., 2015 paper.

- We downloaded and  downsampled to a 100 Million Reads data from the first batch of cells in order to be able to process it interactively in a timely manner.

# Getting started:

We have downloaded all the published data and other files you might need for this and other sessions. In order to get access to these files you can create a softlink to the folder:

```bash
cd ~
ln -s /oasis/tscc/scratch/cshl_2018/ ./
```

Finally, we will make another softlink in your home to your own scratch folder:

```bash
ln -s /oasis/tscc/scratch/ucsd-trainXX/ ./scratch
```

NOTE: Remember to use your own ucsd-train## (i.e. ucsd-train27)

**Location of downsampled fastq files:**

once you have created a softlink for the cshl_2018 folder you should be able to find them here:
```bash
cd ~/cshl_2018/raw_data_macosko
```
 - SRR1853178_1_100M.fastq (read 1)
 - SRR1853178_2_100M.fastq (read 2)

**Organize folders for processing**

I like to keep track of my projects by making a separate folder for each one with the scripts that I used to generate my results (stored in my home) and a folder for the results (which can be quite large and must be stored in scratch). 

```bash
mkdir -p ~/projects/macosko_downsampled_batch_1/scripts/
mkdir -p ~/scratch/projects/macosko_downsampled_batch_1/results/
```

### Interactive Jobs:
For dropseqtools we will be working with interactive jobs in order to visualize on real time the progress of each processing step as we go through the analysis. We have allocated Yeo Lab cluster resources for our use throughout the course. To request an interactive job:
```bash
qsub -I -l walltime=4:00:00 -q home-yeo -l nodes=1:ppn=2
```

Keep in mind that you can adjust your ```walltime=H:MM:SS ``` as necessary.
Two (2) processors (```ppn=2```) should be enough in this case.

### Starting in the right enviroment:
By this point you should have completed the [instructions for downloading and installing dropseqtools 0.13](add link) which means that you should be able to activate your dropseqtools environment

```bash
source activate dropseqtools
```

### (Step 1) Preprocessing:
*The raw reads from the sequencer must be converted into a Picard queryname sorted BAM file for
each library in the sequencer run. Since there are many sequencers and pipelines available to do
this, we leave this step to the user. We use either Picard FastqToSam.*

BAM files are typically known for storing alignment data, or reads that have been aligned to a reference. However, this format can also store reads that are unaligned (just like fastq.gz files), and there are exists some tools (including the ones referenced in this notebook) that use the BAM file format to store unaligned reads.

So before proceeding, we will need to convert fastq.gz files to their BAM file equivalents (in this example, we will call our bam file ```my_unaligned_data.bam``` and will be used as an input into the next step:

```bash
picard FastqToSam \
F1=forward_reads.fastq \
F2=reverse_reads.fastq \
O=my_unaligned_data.bam \
SM=sample001 \
```

**Note:** the backslash (```\```) after each line indicates the continuation of a command on the next line.




#### Tailoring your script to your data

We can take this example and make our own "step 1" script:

Since dropseqtools many steps (at least 13), I like to sequentially number each one of the scripts, inputs and outputs to be able to keep track of things.

Let's then create the first script:
```bash
cd ~/projects/macosko_downsampled_batch_1/scripts/

vi 1_FastqToSam.sh

i 
    
#!/bin/bash

#fastq directory
FD=~/cshl_2018/raw_data_macosko/
#results directory
RD=~/scratch/projects/macosko_downsampled_batch_1/results/

picard FastqToSam \
F1=${FD}SRR1853178_1_100M.fastq \
F2=${FD}SRR1853178_2_100M.fastq \
O=${RD}1_my_unaligned_data.bam \
SM=1_SRR1853178_100M.bam \
2>&1 | tee ${RD}1_my_unaligned_data.bam_log.txt

esc
:wq
    
```

**note:** ```2>&1``` will push both output and error into a file, and ```| tee ${RD}1_my_unaligned_data.bam_log.txt``` will define that file, but also allow users to see the output in the terminal as well.


#### Making your script executable

we have created a script but we now need to make it executable:

The ouptut of ```ls -l``` will show the current permissions for files and folders:

```bash
-rw-r--r-- 1 ucsd-train27 biom262-group 356 Jun 30 15:19 1_FastqToSam.sh
```

The letters **rwx** stand for Read/Write/Execute permission. These rights are shown three times, first for the Owner, then the Group and lastly Others (world)

without an **x** we will not be able to execute our script from the command line, in order to add this feature to the script:

```bash
chmod +x 1_FastqToSam.sh
```

you can double check that this work with ```ls -l```

Now we are ready to execute:
```bash
./1_FastqToSam.sh
```

#### Visualizing a .bam file
.bam files are the binary version of .sam files, so in order to visualize them we can use samtools, specifically it's view option:

```bash
cd ~/scratch/projects/macosko_downsampled_batch_1/results/

samtools view 1_my_unaligned_data.bam | less -S
```

**note** piping your output to less (or head) will allow you to visualize only a small chunk of your .bam file

### Alignment overview:
**GOAL:** Unmapped BAM > aligned and tagged BAM
- (2) Tag cell barcodes
- (3) Tag molecular barcodes
- (4) Filter BAM
- (5) Trim 5’ primer sequence
- (6) Trim 3’ polyA sequence
- (7) SAM > Fastq
- (8) STAR alignment
- (9) Sort STAR alignment in queryname order
- (10) Merge STAR alignment tagged SAM to recover cell/molecular barcodes
- (11) Add gene/exon and other annotation tags
- (12) DetectBeadSynthesisErrors

### Metadata:

- **fasta**: *The reference sequence of the organism. Needed for most aligners.*
- **dict**: A dictionary file as generated by Picard’s CreateSequenceDictionary. Needed for Picard
Tools.*
- **gtf**: *The principle file to determine the location of genomic features like genes, transcripts,
and exons. Many other metadata files we use derive from this original file. We download our
GTF files from ensembl, which has a handy description of the file format here. Ensembl has a
huge number of prepared GTF files for a variety of organisms here.*
- **refFlat**: *This file contains a subset of the the same information in the GTF file in a different
format. Picard tools like the refFlat format, so we require this as well. To make life easy, we
provide a program ConvertToRefFlat that can convert files from GTF format to refFlat for you.*
- **genes.intervals**: *The genes from the GTF file in interval list format. This file is optional, and
useful if you want to go back to your BAM later to see what gene(s) a read aligns to.*
- **exons.intervals**: *The exons from the GTF file in interval list format. This file is optional, and
useful if you want to go back to your BAM and view what exon(s) a read aligns to.*
- **rRNA.intervals**: *The locations of ribosomal RNA in interval list format. This file is optional, but
we find it useful to later assess how much of a dropseq library aligns to rRNA.*
- **reduced.gtf**: *This file contains a subset of the information in the GTF file, but in a far more
human readable format. This file is optional, but can be generated easily by the supplied
ReduceGTF program that will take a GTF file as input.*

### (Step 2) TagBamWithReadsSequenceExtended (tag cells)

*This Drop-seq program extracts bases from the cell/molecular barcode encoding read **(BARCODED_READ)**, and creates a new BAM tag with those bases on the genome read. By default, we use the BAM tag XM for molecular barcodes, and **XC** for cell barcodes, using the **TAG_NAME** parameter.*

*This program is run once per barcode extraction to add a tag. On the first iteration, the cell barcode is
extracted from bases **1-12**. This is controlled by the BASE_RANGE option.*

Example Cell Barcode:

```bash
TagBamWithReadSequenceExtended \
INPUT=my_unaligned_data.bam \
OUTPUT=unaligned_tagged_Cell.bam \
SUMMARY=unaligned_tagged_Cellular.bam_summary.txt \
BASE_RANGE=1-12 \
BASE_QUALITY=10 \
BARCODED_READ=1 \
DISCARD_READ=False \
TAG_NAME=XC \
NUM_BASES_BELOW_QUALITY=1
```

**notes:** don't forget

- to add #!/bin/bash
- to add your variable(s) (i.e. RD=~/scratch/projects/macosko_downsampled_batch_1/results/)
- to add your output log command
    (i.e. 2>&1 | tee ${RD}2_unaligned_tagged_Cellular.bam_summary.txt_log.txt)
- make your script executable

### (Step 3) TagBamWithReadsSequenceExtended (tag umi)

*On the second iteration, the molecular barcode is extracted from bases **13-20** of the barcode read. This program has an option to drop a read **(DISCARD_READ)**, which we use after both barcodes have been extracted, which makes
the output BAM have unpaired reads with additional tags. Additionally, this program has a **BASE_QUALITY** option, which is the minimum base quality of all bases of the barcode being extracted. If more than **NUM_BASES_BELOW_QUALITY** bases falls below this quality, the read pair is discarded.*

Example Molecular Barcode:

```bash
TagBamWithReadSequenceExtended \
INPUT=unaligned_tagged_Cell.bam \
OUTPUT=unaligned_tagged_CellMolecular.bam \
SUMMARY=unaligned_tagged_Molecular.bam_summary.txt \
BASE_RANGE=13-20 \
BASE_QUALITY=10 \
BARCODED_READ=1 \
DISCARD_READ=True \
TAG_NAME=XM \
NUM_BASES_BELOW_QUALITY=1
```

### (Step 4) FilterBam

*This Drop-seq program is used to remove reads where the cell or molecular barcode has low quality
bases. During the run of **TagBamWithReadSequenceExtended**, an **XQ** tag is added to each read to
represent the number of bases that have quality scores below the **BASE_QUALITY** threshold. These
reads are then removed from the BAM.*

Example:
```bash
FilterBAM \
TAG_REJECT=XQ \
INPUT=unaligned_tagged_CellMolecular.bam \
OUTPUT=unaligned_tagged_filtered.bam \
```

### (Step 5) TrimStartingSequence

*This Drop-seq program is one of two sequence cleanup programs designed to trim away any extra
sequence that might have snuck it’s way into the reads. In this case, we trim the SMART Adapter that
can occur 5’ of the read. In our standard run, we look for at least 5 contiguous bases **(NUM_BASES)** of
the SMART adapter **(SEQUENCE)** at the 5’ end of the read with no errors **(MISMATCHES)** , and hard
clip those bases off the read.*


Example:
```bash
TrimStartingSequence \
INPUT=unaligned_tagged_filtered.bam \
OUTPUT=unaligned_tagged_trimmed_smart.bam \
OUTPUT_SUMMARY=adapter_trimming_report.txt \
SEQUENCE=AAGCAGTGGTATCAACGCAGAGTGAATGGG \
MISMATCHES=0 \
NUM_BASES=5
```

### (Step 6) PolyATrimmer

*This Drop-seq program is the second sequence cleanup program designed to trim away trailing polyA
tails from reads. It searches for at least **6 (NUM_BASES)** contiguous A’s in the read with **0 (MISMATCHES)**, and hard clips the read to remove these bases and all bases 3’ of the polyA run.*

Example:
```bash
PolyATrimmer \
INPUT=unaligned_tagged_trimmed_smart.bam \
OUTPUT=unaligned_mc_tagged_polyA_filtered.bam \
OUTPUT_SUMMARY=polyA_trimming_report.txt \
MISMATCHES=0 \
NUM_BASES=6

```

### (Step 7) SamToFastq

*Now that your data has had the cell and molecular barcodes extracted, the reads have been cleaned
of SMARTSeq primer and polyA tails, and the data is now unpaired reads, it’s time to align. To do this,
we extract the FASTQ files using Picard’s **SamToFastq** program.*

```bash
picard SamToFastq \
INPUT=unaligned_mc_tagged_polyA_filtered.bam \
FASTQ=unaligned_mc_tagged_polyA_filtered.fastq
```

### (Step 8) Alignment - STAR

*We use STAR as our RNA aligner. The manual for STAR can be found [here](https://github.com/alexdobin/STAR/blob/master/doc/STARmanual.pdf). There are many potential
aligners one could use at this stage, and it’s possible to substitute in your lab’s favorite. We haven’t
tested other aligners in methodical detail, but all should produce valid BAM files that can be plugged
into the rest of the process detailed here.
If you’re unsure how to create an indexed reference for STAR, please read the STAR manual.
Below is a minimal invocation of STAR. Since STAR contains a huge number of options to tailor
alignment to a library and trade off sensitivity vs specificity, you can alter the default settings of the
algorithm to your liking, but we find the defaults work reasonably well for Drop-seq. Be aware that
STAR requires roughly 30 gigabytes of memory to align a single human sized genome, and 60
gigabytes
for our human/mouse reference.*

Example:
```bash
/path/to/STAR/STAR \
--genomeDir /path/to/STAR_REFERENCE \
--readFilesIn unaligned_mc_tagged_polyA_filtered.fastq \
--outFileNamePrefix star
```

**note:** path to star reference is  ```~/cshl_2018/reference_files/mm10_star/```

### (Step 9) SortSam

*This picard program is invoked after alignment, to guarantee that the output from alignment is sorted
in queryname order. As a side bonus, the output file is a BAM (compressed) instead of SAM
(uncompressed.)*

Example:

```bash
picard SortSam \
I=starAligned.out.sam \
O=aligned.sorted.bam \
SO=queryname
```

### (Step 10) MergeBamAlignment

*This Picard program merges the sorted alignment output from STAR **(ALIGNED_BAM)** with the
unaligned BAM that had been previously tagged with molecular/cell barcodes **(UNMAPPED_BAM)**.
This recovers the BAM tags that were “lost” during alignment. The **REFERENCE_SEQUENCE** argument
refers to the fasta metadata file.
We ignore secondary alignments, as we want only the best alignment from STAR (or another aligner),
instead of assigning a single sequencing read to multiple locations on the genome.*

Example:
```bash
picard MergeBamAlignment \
REFERENCE_SEQUENCE=my_fasta.fasta \
UNMAPPED_BAM=unaligned_mc_tagged_polyA_filtered.bam \
ALIGNED_BAM=aligned.sorted.bam \
OUTPUT=merged.bam \
INCLUDE_SECONDARY_ALIGNMENTS=false \
PAIRED_RUN=false
```

### (Step 11) TagReadWithGeneExon

*This is a Drop-seq program that adds a BAM tag “GE” onto reads when the read overlaps the exon of a
gene. This tag contains the name of the gene, as reported in the annotations file. You can use either a
GTF or a RefFlat annotation file with this program, depending on what annotation data source you
find most useful. This is used later when we extract digital gene expression (DGE) from the BAM.*

Example:
```bash
TagReadWithGeneExon \
I=merged.bam \
O=star_gene_exon_tagged.bam \
ANNOTATIONS_FILE=${refFlat} \
TAG=GE
```

**note:** path to annotation file is  ```~/cshl_2018/reference_files/mm10/mm10.refFlat``` remember you can creat a variable

### (Step 12) DetectBeadSynthesisErrors

*In June 2015, we noticed that a recently purchased batch of ChemGenes beads generated a
population of cell barcodes (about 10-20%) with sequences that shared the first 11 bases, but differed
at the last base. These same cell barcodes also had a very high percentage of the base “T” at the last
position of the UMI. Based on these observations, we concluded that a percentage of beads in the lot
had not undergone all twelve split-and-pool bases (perhaps they had stuck to some piece of
equipment or container, and the been re-introduced after the missing synthesis cycle). Thus, the
20-bp Read 1 contained a mixed base at base 12 (in actuality, the first base of the UMI) and a fixed
T-base at base 20 (in actuality, the first base of the polyT segment).*

*To correct for this, we generated **DetectBeadSynthesisErrors**, which identifies cell barcodes with
aberrant “fixed” UMI bases. If only the last UMI base is fixed as a T, the cell barcode is corrected (the
last base is trimmed off) and all cell barcodes with identical sequence at the first 11 bases are merged
together. If any other UMI base is fixed, the reads with that cell barcode are discarded.
The program asks the user to select a number of barcodes on which to perform the correction. We
use roughly 2 times the anticipated number cells, as we empirically found that this allows us to
recover nearly every defective cell barcode that corresponds to a STAMP (rather than an empty bead
cell barcode).*

Example:

```bash
DetectBeadSynthesisErrors \
I=my.bam \
O=my_clean.bam \
OUTPUT_STATS=my.synthesis_stats.txt \
SUMMARY=my.synthesis_stats.summary.txt \
NUM_BARCODES= <roughly 2x the number of cells> \
PRIMER_SEQUENCE=AAGCAGTGGTATCAACGCAGAGTAC \
```

*This program reads in the BAM file, and looks at the distribution of bases at each position of all UMIs
for a cell barcode. It detects unusual distributions of base frequency, where a base with >=80%
frequency at any position is detected as an error. Barcodes with less than 25 total UMIs are ignored.
There are a number of different errors that are categorized:*

*- **SYNTHESIS_MISSING_BASE** - 1 or more bases missing from cell barcode, resulting in fixed T’s
at the end of UMIs. This counts the maximum number of fixed sequential T’s in the UMIs at
the end. This error type is cleaned up by the software for situations where there is a single
base missing, and is by far the most common error. The fix involves inserting an “N” base
before the last cell barcode base, effectively shifting the reading frame back to where it
should be. This will both collapse these beads back together in further analysis, as well as
repair the UMIs for these bead barcodes.*

![title](img/synthesis_missing_base.png)

*- **SINGLE_UMI_ERROR** - At each position of the UMIs, the base distribution is highly skewed, i.e.
at each position, a single base appears in >= 80% of the UMIs for that cell. There’s no fix for
this currently. Cell barcodes with this property are dropped. These cells have the interesting
property that the number of genes and transcripts are at a close to 1:1 ratio, as there’s
generally only 1 UMI for every gene.*

*- **PRIMER_MATCH** - Same as SINGLE_UMI_ERROR, but in addition the UMI perfectly matches
one of the PCR primers. These cell barcodes are dropped. These errors are only detected if a
PRIMER_SEQUENCE argument is supplied.*

*- **OTHER** - UMIs are extremely skewed towards at least one base (and not T at the last base),
but not at all 8 positions. These cell barcodes are dropped.*


#### The file my.synthesis_stats.txt contains a bunch of useful information:

*- **CELL_BARCODE** - the 12 base cell barcode*

*- **NUM_UMI** - the number of total umis observed*

*- **FIRST_BIASED_BASE** - the first base position where any bias is observed. ­1 for no detected
bias*

*- **SYNTH_MISSING_BASE** - as #3 but specific to runs of T’s at the end of the UMI*

*- **ERROR_TYPE** - see error type definitions above*

*- For bases 1-8 of the UMI, the observed base counts across all UMIs. This is a “|” delimited
field, with counts of the A,C,G,T,N bases.*

*The file my.synthesis_stats.summary.txt contains a histogram of the SYNTHESIS_MISSING_BASE
errors, as well as the counts of all other errors, the number of total barcodes evaluated, and the
number of barcodes ignored.*

### Digital Gene Expression

*Extracting Digital Gene Expression (DGE) data from an aligned library is done using the Drop­seq
program DigitalExpression. The input to this program is the aligned BAM from the alignment
workflow. There are two outputs available: the primary is the DGE matrix, with each a row for each
gene, and a column for each cell. The secondary analysis is a summary of the DGE matrix on a per­cell
level, indicating the number of genes and transcripts observed.*

![title](img/DGE.png)

Example:

```bash
DigitalExpression \
I=out_gene_exon_tagged.bam \
O=out_gene_exon_tagged.dge.txt.gz \
SUMMARY=out_gene_exon_tagged.dge.summary.txt \
NUM_CORE_BARCODES=100
```

### Final sorting and filter in preparation for QC

in order to facilitate QC of our data we will have to sort and filter the final BAM file (step 12 output) for this:

```bash
samtools sort -@ 4 input_file.bam -o output_file.sorted.bam

samtools index output_file.sorted.bam
```